In [204]:
import pandas as pd
import sqlite3
import shlex
import numpy as np
import collections

def auto_cast(s):
    try:
        return int(s)
    except ValueError:
        try:
            return float(s)
        except ValueError:
            if s.lower() in ["false", "true"]:
                try:
                    return bool(s)
                except ValueError:
                    pass
    return str(s)

def cmd_parser(s, sort=False, template=None):
    if '.py' in s:
        s = s.split('.py')[1]
    res = [x.split('=') if '=' in x else x.split(' ') for x in s.split(' --')[1:]]
    res = dict([(x[0], auto_cast(x[1])) if len(x) == 2 else (x[0], True) for x in res])
    if template is not None:
        template_dict = dict([(x, np.nan) for x in template])
        template_dict.update(res)
        res = template_dict
    if sort:
        return collections.OrderedDict(sorted(res.items()))
    else:
        return res



In [205]:
# Establish DB connection
conn = sqlite3.connect("/usr/stud/plapp/runnn/runnn_data")
df = pd.read_sql_query("select * from tasks where status is 'done';", conn)

In [206]:
# Get list of hyper params
params = []
for row in df.cmd:
    params += ([x.split('=')[0] if '=' in x else x.split(' ')[0] for x in row.split(' --')[1:]])
params = np.unique(params).astype(str)  # is already sorted

for param in params:
    df[param] = np.nan

In [207]:
# Split args into columns
cmd_splitter = lambda x: pd.Series([i for i in cmd_parser(x, True, params).values()])
df[params] = df.cmd.apply(cmd_splitter)

In [208]:
# Remove NaN scores
# df = df.dropna(thresh=len(params))

In [233]:
# Get list of datasets
datasets = [x for x in df.dataset.unique() if x == x]

In [234]:
print 'LEADERBOARD'
for dataset in datasets:
    score_mean = df[df.dataset == dataset].score.mean() * 100
    score_std = df[df.dataset == dataset].score.std() * 100
    print "%s:\t\t%.2f%%\t+/- %.2f" % (dataset[:6], score_mean, score_std)

LEADERBOARD
stl10:		39.92%	+/- 4.37
cifar_:		29.96%	+/- 1.76
frgc:		43.33%	+/- 3.33
